# Event Planning Agent


If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Crew, Task

- Create a .env file and add your keys as shown below
``` 
OPENAI_API_KEY=your_openai_key_here
OPENAI_MODEL_NAME=gpt-3.5-turbo
SERPER_API_KEY=your_serper_key_here
```

- Below code will load your keys from .env file

In [4]:
from dotenv import load_dotenv
load_dotenv()  # This will load the variables from the .env file

import os

openai_api_key = os.getenv("OPENAI_API_KEY")
model_name = os.getenv("OPENAI_MODEL_NAME")
serper_api_key = os.getenv("SERPER_API_KEY")

## crewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [6]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [7]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [8]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [9]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [10]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [11]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [12]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [13]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [17]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for the Tech Innovation Conference. I should consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "tech event venues San Francisco"} 


Search results: Title: Networking Venues for Rent in San Francisco, CA - Tagvenue
Link: https://www.tagvenue.com/us/hire/networking-venues/san-francisco
Snippet: SoMa (South of Market): Known for its tech and startup scene, SoMa provides modern and creative spaces ideal for tech networking events. Attractions like ...
---
Title: The 25 Best Event Venues in San Francisco - Bizzabo
Link: https://www.bizzabo.com/blog/san-francisco-event-venues
Snippet: Bespoke, located in the heart of San Francisco's tech and retail hub, is a un

This is the agent final answer: The 25 Best Event Venues in San Francisco | Bizzabo
City View at METREON
Location: SoMa
Capacity: 2,000
City View at Metreon in San Francisco is a versatile event venue. Newly renovated to offer both flexibility and upscale ambiance, this San Francisco event space features more than 31,000 square feet of space that can be adjusted to accommodate various events, ranging from intimate gatherings of 200 to large-scale events for up to 2,000 people. 
Its key attractions include floor-to-ceiling windows providing an unobstructed view of San Francisco’s skyline and an expansive outdoor terrace, offering a unique open-air experience. This makes it an ideal venue for receptions, conferences, and other special occasions. 

The Village Venue
Location: Mid-Market
Capacity: 1,100
The Village, located at 969 Market Street in San Francisco, is a versatile and fully serviced event venue. It offers a creative and dynamic space of more than 17,000 square feet for various

Final Answer: The 25 Best Event Venues in San Francisco | Bizzabo
City View at METREON
Location: SoMa
Capacity: 2,000
City View at Metreon in San Francisco is a versatile event venue. Newly renovated to offer both flexibility and upscale ambiance, this San Francisco event space features more than 31,000 square feet of space that can be adjusted to accommodate various events, ranging from intimate gatherings of 200 to large-scale events for up to 2,000 people. 
Its key attractions include floor-to-ceiling windows providing an unobstructed view of San Francisco’s skyline and an expansive outdoor terrace, offering a unique open-air experience. This makes it an ideal venue for receptions, conferences, and other special occasions. 

The Village Venue
Location: Mid-Market
Capacity: 1,100
The Village, located at 969 Market Street in San Francisco, is a versatile and fully serviced event venue. It offers a creative and dynamic space of more than 17,000 square feet for various events. 
With in-

Exception in thread Thread-14 (_execute):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/jilsonjoseph/Desktop/projects/secondbrain/agents/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jilsonjoseph/Desktop/projects/secondbrain/agents/.venv/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/Users/jilsonjoseph/Desktop/projects/secondbrain/agents/.venv/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jilsonjoseph

Thought: I now know the final answer
Final Answer: The Tech Innovation Conference will be held at City View at METREON, a newly renovated event venue in San Francisco. The venue offers over 31,000 square feet of divisible space that can accommodate 200 to 2,000 guests, making it suitable for various events. Amenities include bride's and groom's dressing areas, coat check, and a panoramic view of San Francisco's skyline. To secure a date for the conference, it is recommended to contact City View at METREON directly or visit their website for booking availability.

> Finished chain.


- Display the generated `venue_details.json` file.

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'SoMa',
 'booking_status': 'Available',
 'capacity': 2000,
 'name': 'City View at METREON'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [19]:
from IPython.display import Markdown
Markdown("marketing_report.md")

### Marketing Activities Report for Tech Innovation Conference

#### Marketing Strategies:
1. Utilize unique entertainment options such as drone shows and live art to make the conference unforgettable.
2. Offer early bird discounts and optimize the event registration website to drive registrations.
3. Leverage social media, email campaigns, and online event calendars for promotion.
4. Implement event gamification and survey the target audience for feedback.
5. Use innovative event marketing strategies to attract attention and generate interest.
6. Curate learning sessions, provide insights from industry experts, and offer hands-on experiences with event tech.
7. Join the Marketing Strategy & Technology Summit to explore the latest trends in digital marketing.

#### Attendee Engagement Strategies:
1. Combine innovative technology with personalized experiences for long-lasting impact.
2. Incorporate AI-driven personalization and immersive virtual experiences to engage with audiences.
3. Utilize interactive polls, Q&A sessions, and real-time surveys for attendee engagement.
4. Provide early registrants with exclusive discount codes for upcoming events.
5. Offer interactive activities, networking opportunities, educational sessions, and entertainment for engagement.
6. Conduct hands-on workshops and demonstrations to allow attendees to practice new concepts immediately.

By implementing these marketing and attendee engagement strategies, we aim to achieve maximum exposure and participation at the Tech Innovation Conference.